In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import tensorflow as tf
from seq2seq import models
from seq2seq.data import data_utils, vocab
from seq2seq.training import utils as training_utils
from seq2seq.training import HParamsParser

In [3]:
VOCAB_SOURCE = "/tmp/toydata/train/vocab.txt"
VOCAB_TARGET = "/tmp/toydata/train/vocab.txt"
MODEL = "AttentionSeq2Seq"
MODEL_DIR = "/Users/dennybritz/Downloads/toy"
INPUT_FILE = "/tmp/toydata/test/sources.txt"
BATCH_SIZE = 32

In [4]:
# Load vocabulary
source_vocab_info = vocab.get_vocab_info(VOCAB_SOURCE)
target_vocab_info = vocab.get_vocab_info(VOCAB_TARGET)

In [5]:
# Find model class
model_class = getattr(models, MODEL)

# Parse parameter and merge with defaults
hparams = model_class.default_params()
hparams_parser = HParamsParser(hparams)
saved_hparams = training_utils.read_hparams(os.path.join(MODEL_DIR, "hparams.txt"))
hparams = hparams_parser.parse(saved_hparams)

In [6]:
# Create model
model = model_class(
    source_vocab_info=source_vocab_info,
    target_vocab_info=target_vocab_info,
    params=hparams)
featurizer = model.create_featurizer()

In [7]:
def model_fn(features, labels, params, mode):
  """Builds the model graph"""
  return model(features, labels, params, mode)
  
# Create estimator
estimator = tf.contrib.learn.estimator.Estimator(model_fn=model_fn, model_dir=MODEL_DIR)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, 'tf_random_seed': None, '_task_id': 0, 'save_checkpoints_steps': None, '_is_chief': True, 'keep_checkpoint_every_n_hours': 10000, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x111828fd0>, 'keep_checkpoint_max': 5, 'save_summary_steps': 100, '_evaluation_master': '', '_environment': 'local', 'save_checkpoints_secs': 600, '_master': '', '_task_type': None, '_num_ps_replicas': 0}


In [8]:
# Create input function for input file
data_provider = lambda: data_utils.make_parallel_data_provider(
        [INPUT_FILE], [INPUT_FILE], shuffle=False)
input_fn = training_utils.create_input_fn(
      data_provider,
      featurizer,
      BATCH_SIZE)

In [12]:
predictions = estimator.predict(input_fn=input_fn)
predictions_ = next(predictions)

INFO:tensorflow:Creating vocabulary lookup table of size 20
INFO:tensorflow:Creating vocabulary lookup table of size 20
Tensor("model/att_seq2seq/embedding_lookup_1:0", shape=(32, 100), dtype=float32)
Tensor("model/att_seq2seq/attention_decoder_1/RNN/while/rnn_step/fixed_decoder_inputs/embedding_lookup:0", shape=(32, 100), dtype=float32)
INFO:tensorflow:Loading model from checkpoint: /Users/dennybritz/Downloads/toy/model.ckpt-1-?????-of-00001.


In [22]:
with open(VOCAB_TARGET) as file:
  vocab_lookup = [l.strip() for l in file.readlines()]
" ".join([vocab_lookup[i] for i in predictions_["predictions"]])

'7 7 13 12 12 12 5 12 4 4 4 4 12 12 12'